In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from scipy.stats import t
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, RidgeClassifier
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Lets try out some stuff on a super small dataframe

In [ ]:
# intialise data of lists. 
data = {'Name':['Tom', 'nick', 'krish', 'nan'], 'Age':[np.nan, 21, 19, 22], 'Sex':['M', 'M', 'F', 'M'], 'Footwear':['Sandals', 'Shoes', 'Sandals', 'Shoes']} 
  
# Create DataFrame 
df_sample = pd.DataFrame(data) 
  
# Print the output. 
df_sample

### get rid of nulls

In [ ]:
imp = SimpleImputer(strategy='most_frequent')

In [ ]:
imp.fit(df_sample)

In [ ]:
df_imp = imp.transform(df_sample)

In [ ]:
df_imp

### split the dataframe into X and y

In [ ]:
X_df = df_sample['Footwear']
X_df

In [ ]:
y_df = df_sample.drop(['Footwear'], axis=1)
y_df

In [ ]:
# Get dummies
df_sample_dummies = pd.get_dummies(y_df, prefix_sep='_', drop_first=True)

# X head
print(df_sample_dummies)

In [ ]:
# Get sparse dummies
df_sample_sparse_dummies = pd.get_dummies(df_sample, prefix_sep='_', drop_first=True, sparse=True)

# X head
print(df_sample_sparse_dummies)

In [ ]:
df_sample_dummies.info()

In [ ]:
df_sample_sparse_dummies.info()

In [ ]:
from sklearn.cluster import KMeans
# dictionary to fill with the single square errors
sse = {}
# Fit KMeans and calculate SSE for each k
for k in range(1, 5):
  
    # Initialize KMeans with k clusters
    kmeans = KMeans(n_clusters=k, random_state=1)
    
    # Fit KMeans on the normalized dataset
    kmeans.fit(df_sample_sparse_dummies)
    
    # Assign sum of squared distances to k element of dictionary
    sse[k] = kmeans.inertia_ 
# Add the plot title "The Elbow Method"
plt.title('The Elbow Method')

# Add X-axis label "k"
plt.xlabel('k')

# Add Y-axis label "SSE"
plt.ylabel('SSE')

# Plot SSE values for each key in the dictionary
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

In [ ]:
pca = PCA(n_components=2)
y_pca = pca.fit_transform(df_sample_sparse_dummies)

In [ ]:
y_pca

# Let's try it on the big dataset

In [3]:
df_read = pd.read_csv('Data/Loan_Storied.csv',low_memory=False,index_col=0, 
                parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 
                               'debt_settlement_flag_date', 'settlement_date','sec_app_earliest_cr_line'])

C:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df = df_read.copy()

In [5]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate_pct,installment,grade,sub_grade,emp_title,emp_length_years,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util_pct,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,meets_credit_policy
0,5000.000,5000.000,4975.000,36.000,0.106,162.870,B,B2,NaN,10.000,RENT,24000.000,Verified,2011-12-01,Fully Paid,n,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.650,0.000,1985-01-01,1.000,nan,nan,3.000,0.000,13648.000,0.837,9.000,f,0.000,0.000,5863.155,5833.840,5000.000,863.160,0.000,0.000,0.000,2015-01-01,171.620,NaT,2019-05-01,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,N,NaT,NaN,NaT,nan,nan,nan,True
1,2500.000,2500.000,2500.000,60.000,0.153,59.830,C,C4,Ryder,0.500,RENT,30000.000,Source Verified,2011-12-01,Charged Off,n,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.000,0.000,1999-04-01,5.000,nan,nan,3.000,0.000,1687.000,0.094,4.000,f,0.000,0.000,1014.530,1014.530,456.460,435.170,0.000,122.900,1.110,2013-04-01,119.660,NaT,2016-10-01,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,N,NaT,NaN,NaT,nan,nan,nan,True
2,2400.000,2400.000,2400.000,36.000,0.160,84.330,C,C5,NaN,10.000,RENT,12252.000,Not Verified,2011-12-01,Fully Paid,n,NaN,small_business,real estate business,606xx,

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422722 entries, 0 to 1422721
Columns: 142 entries, loan_amnt to meets_credit_policy
dtypes: bool(1), datetime64[ns](8), float64(108), object(25)
memory usage: 1.5+ GB


We need to get rid of columns that contain information that happens after a loan has been given out, such as hardships.  This is really an important step and it also helps us downsize our dataset a little bit.  

In [7]:
drop_list = ['collection_recovery_fee', 'debt_settlement_flag','debt_settlement_flag_date','deferral_term','delinq_amnt','hardship_amount','hardship_dpd', 'hardship_end_date', 'hardship_flag','hardship_last_payment_amount','hardship_length', 'hardship_loan_status','hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'last_pymnt_amnt','last_pymnt_d','next_pymnt_d','orig_projected_additional_accrued_interest','out_prncp','out_prncp_inv','payment_plan_start_date','pymnt_plan','recoveries','settlement_amount','settlement_date','settlement_percentage','settlement_status','settlement_term','total_pymnt','total_pymnt_inv','total_rec_int','total_rec_late_fee','total_rec_prncp','desc','zip_code','title','emp_title','meets_credit_policy']
df = df.drop(drop_list, axis=1)

### And now lets change the dates to floats by subtracting to or from the issue date

In [8]:
df['issue_d_minus_earliest_cr_line'] = (df['issue_d']-df['earliest_cr_line']).dt.days
df['last_credit_pull_d_minus_issue_d'] = (df['last_credit_pull_d']-df['issue_d']).dt.days
df['issue_d_minus_sec_app_earliest_cr_line'] = (df['issue_d']-df['sec_app_earliest_cr_line']).dt.days

### Drop the date fields

In [9]:
dt_drop_list = ['issue_d','earliest_cr_line','last_credit_pull_d','sec_app_earliest_cr_line']
df = df.drop(dt_drop_list, axis=1)
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate_pct,installment,grade,sub_grade,emp_length_years,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util_pct,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,issue_d_minus_earliest_cr_line,last_credit_pull_d_minus_issue_d,issue_d_minus_sec_app_earliest_cr_line
0,5000.000,5000.000,4975.000,36.000,0.106,162.870,B,B2,10.000,RENT,24000.000,Verified,Fully Paid,credit_card,AZ,27.650,0.000,1.000,nan,nan,3.000,0.000,13648.000,0.837,9.000,f,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,9830.000,2708.000,nan
1,2500.000,2500.000,2500.000,60.000,0.153,59.830,C,C4,0.500,RENT,30000.000,Source Verified,Charged Off,car,GA,1.000,0.000,5.000,nan,nan,3.000,0.000,1687.000,0.094,4.000,f,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4627.000,1766.000,nan
2,2400.000,2400.000,2400.000,36.000,0.160,84.330,C,C5,10.000,RENT,12252.000,Not Verified,Fully Paid,small_business,IL,8.720,0.000,2.000,nan,nan,2.000,0.000,2956.000,0.985,10.000,f,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3682.000,2009.000,nan
3,10000.000,10000.000,10000.000,36.000,0.135,339.310,C,C1,10.000,RENT,49200.000,Source Verified,Fully Paid,other,CA,20.000,0.000,1.000,35.000,nan,10.000,0.000,5598.000,0.210,37.000,f,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,5782.000,1583.000,nan
4,3000.000,3000.000,3000.000,60.000,0.127,67.790,B,B5,1.000,RENT,80000.000,Source Verified,Fully Paid,other,OR,17.940,0.000,0.000,38.000,nan,15.000,0.000,27783.000,0.539,38.000,f,0.000,nan,1.000,Individual,nan,nan,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,nan,nan,nan,

### Convert Default to Charged Off since it is the same outcome 

In [10]:
df['loan_status'] = np.where(df['loan_status'] == 'Default','Charged Off',df['loan_status'])
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

# Get rid of null values

In [11]:
df.isnull().sum()

loan_amnt                                       0
funded_amnt                                     0
funded_amnt_inv                                 0
term_months                                     0
int_rate_pct                                    0
installment                                     0
grade                                           0
sub_grade                                       0
emp_length_years                            84136
home_ownership                                  0
annual_inc                                      4
verification_status                             0
loan_status                                     0
purpose                                         0
addr_state                                      0
dti                                           461
delinq_2yrs                                    29
inq_last_6mths                                 30
mths_since_last_delinq                     717293
mths_since_last_record                    1180462


In [12]:
imp_con = SimpleImputer(strategy='constant',fill_value='Not Verified')

In [13]:
df.loc[:,'verification_status_joint'] = imp_con.fit_transform(df.loc[:,'verification_status_joint'].to_numpy().reshape(-1,1))

In [14]:
imp_mean = SimpleImputer()

In [15]:
df.loc[:,'emp_length_years'] = imp_mean.fit_transform(df.loc[:,'emp_length_years'].to_numpy().reshape(-1,1))

In [16]:
df.loc[:,'annual_inc'] = imp_mean.fit_transform(df.loc[:,'annual_inc'].to_numpy().reshape(-1,1))

In [17]:
df.iloc[:,15:25] = imp_mean.fit_transform(df.iloc[:,15:25])

In [18]:
df.iloc[:,26:28] = imp_mean.fit_transform(df.iloc[:,26:28])

In [19]:
df.iloc[:,30:32] = imp_mean.fit_transform(df.iloc[:,30:32])

In [20]:
df.iloc[:,33:100] = imp_mean.fit_transform(df.iloc[:,33:100])

In [21]:
df.isnull().sum().sum()

0

In [22]:
df.isnull().values.sum()

0

### Let's use a Heatmap and see if we can get rid of correlated data

Here we are creating a quick function that will create a heatmap. This heat map will show correlation between columns. 

In [23]:
def heatMap(df, mirror=False):

   # Create Correlation df
   corr = df.corr()
   # Plot figsize
   fig, ax = plt.subplots(figsize=(100, 100))
   # Generate Color Map
   colormap = sns.diverging_palette(220, 10, as_cmap=True)
   
   if mirror == True:
      #Generate Heat Map, allow annotations and place floats in map
      sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
      #Apply xticks
      plt.xticks(range(len(corr.columns)), corr.columns);
      #Apply yticks
      plt.yticks(range(len(corr.columns)), corr.columns)
      #show plot

   else:
      # Drop self-correlations
      dropSelf = np.zeros_like(corr)
      dropSelf[np.triu_indices_from(dropSelf)] = True
      # Generate Color Map
      colormap = sns.diverging_palette(220, 10, as_cmap=True)
      # Generate Heat Map, allow annotations and place floats in map
      sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f", mask=dropSelf)
      # Apply xticks
      plt.xticks(range(len(corr.columns)), corr.columns);
      # Apply yticks
      plt.yticks(range(len(corr.columns)), corr.columns)
   # show plot
   plt.show()

In [ ]:
heatMap(df)

In [24]:
corr_drop_list = ['funded_amnt','funded_amnt_inv','installment','open_acc','tot_cur_bal','total_bal_il','num_rev_tl_bal_gt_0','mo_sin_old_rev_tl_op']
df = df.drop(corr_drop_list, axis=1)

In [ ]:
heatMap(df)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422722 entries, 0 to 1422721
Data columns (total 92 columns):
loan_amnt                                 1422722 non-null float64
term_months                               1422722 non-null float64
int_rate_pct                              1422722 non-null float64
grade                                     1422722 non-null object
sub_grade                                 1422722 non-null object
emp_length_years                          1422722 non-null float64
home_ownership                            1422722 non-null object
annual_inc                                1422722 non-null float64
verification_status                       1422722 non-null object
loan_status                               1422722 non-null object
purpose                                   1422722 non-null object
addr_state                                1422722 non-null object
dti                                       1422722 non-null float64
delinq_2yrs            

# These are the object datatype variables that will need to be changed to numeric variables.  

In [26]:
obj_list = sorted(list(df.select_dtypes(include=['object']).columns))
for i in range(len(obj_list)):
    print('column name:  ',obj_list[i])
    print('number of unique values:  ',len(df[obj_list[i]].unique()))
    print('unique values:  ')
    print(df[obj_list[i]].unique())
    print("--------------------------")

column name:   addr_state
number of unique values:   51
unique values:  
['AZ' 'GA' 'IL' 'CA' 'OR' 'NC' 'TX' 'VA' 'MO' 'CT' 'UT' 'FL' 'NY' 'PA'
 'MN' 'NJ' 'KY' 'OH' 'SC' 'RI' 'LA' 'MA' 'WA' 'WI' 'AL' 'CO' 'KS' 'NV'
 'AK' 'MD' 'WV' 'VT' 'MI' 'DC' 'SD' 'NH' 'AR' 'NM' 'MT' 'HI' 'WY' 'OK'
 'DE' 'MS' 'TN' 'IA' 'NE' 'ID' 'IN' 'ME' 'ND']
--------------------------
column name:   application_type
number of unique values:   2
unique values:  
['Individual' 'Joint App']
--------------------------
column name:   grade
number of unique values:   7
unique values:  
['B' 'C' 'A' 'E' 'F' 'D' 'G']
--------------------------
column name:   home_ownership
number of unique values:   6
unique values:  
['RENT' 'OWN' 'MORTGAGE' 'OTHER' 'NONE' 'ANY']
--------------------------
column name:   initial_list_status
number of unique values:   2
unique values:  
['f' 'w']
--------------------------
column name:   loan_status
number of unique values:   2
unique values:  
['Fully Paid' 'Charged Off']
--------------

In [27]:
df = pd.get_dummies(df, prefix_sep='_', drop_first=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422722 entries, 0 to 1422721
Columns: 197 entries, loan_amnt to verification_status_joint_Verified
dtypes: float64(82), uint8(115)
memory usage: 1.0 GB


In [29]:
df.dtypes

loan_amnt                                    float64
term_months                                  float64
int_rate_pct                                 float64
emp_length_years                             float64
annual_inc                                   float64
dti                                          float64
delinq_2yrs                                  float64
inq_last_6mths                               float64
mths_since_last_delinq                       float64
mths_since_last_record                       float64
pub_rec                                      float64
revol_bal                                    float64
revol_util_pct                               float64
total_acc                                    float64
collections_12_mths_ex_med                   float64
mths_since_last_major_derog                  float64
policy_code                                  float64
annual_inc_joint                             float64
dti_joint                                    f

# Split the dataset into X and y datasets

In [31]:
df_y = df['loan_status_Fully Paid']
df_y.head()

0    1
1    0
2    1
3    1
4    1
Name: loan_status_Fully Paid, dtype: uint8

In [32]:
y = np.ravel(df_y.to_numpy().reshape(-1,1))

In [33]:
y.shape

(1422722,)

In [34]:
X_drop_list = ['loan_status_Fully Paid']
df_X = df.drop(X_drop_list, axis=1)
df_X.head()

,loan_amnt,term_months,int_rate_pct,emp_length_years,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,revol_util_pct,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,mo_sin_old_il_acct,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,issue_d_minus_earliest_cr_line,last_credit_pull_d_minus_issue_d,issue_d_minus_sec_app_earliest_cr_line,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,application_type_Joint App,verification_status_joint_Source Verified,verification_status_joint_Verified
0,5000.000,36.000,0.106,10.000,24000.000,27.650,0.000,1.000,34.286,70.542,0.000,13648.000,0.837,9.000,0.000,43.730,1.000,117626.681,18.899,0.000,249.100,1.039,2.787,0.766,1.721,19.814,71.199,1.408,2.980,5543.128,58.085,32906.673,1.084,1.592,2.284,4.686,13487.702,10284.598,59.715,0.000,125.747,13.171,7.880,1.660,23.870,39.597,6.733,35.780,0.510,3.643,5.640,4.736,8.063,8.558,8.276,14.555,11.638,0.001,0.003,0.089,2.175,94.147,44.872,0.000,0.000,174616.241,49752.641,21733.868,42261.115,31407.695,0.743,1.646,11.314,57.187,2.954,12.760,0.055,0.087,36.724,9830.000,2708.000,5467.682,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2500.000,60.000,0.153,0.500,30000.000,1.000,0.000,5.000,34.

In [35]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422722 entries, 0 to 1422721
Columns: 196 entries, loan_amnt to verification_status_joint_Verified
dtypes: float64(82), uint8(114)
memory usage: 1.0 GB


In [36]:
X = df_X.to_numpy()
X.shape

(1422722, 196)

### Let's split it into Testing and Training datasets
We want to split the data two times.  We want a dataset for training the models and selecting hyperparameters, then another dataset for testing which can be used to compare the models against one another, and then a final dataset for testing our slected model.  Since we need three groups of data we will split the data twice.  

In [37]:
X_int_train, X_final_test, y_int_train, y_final_test =train_test_split(X, y, test_size=0.2,random_state=1)

In [38]:
X_train, X_model_test, y_train, y_model_test =train_test_split(X_int_train, y_int_train, test_size=0.2,random_state=1)

In [39]:
clf = RidgeClassifierCV(alphas=np.logspace(-6, 6, 13),
                        normalize=True).fit(X_train, y_train)

In [40]:
clf.score(X_train, y_train)

0.8056045801342279

In [41]:
clf.alpha_

0.001

In [43]:
y_pred_prob = clf.predict_proba(X_test)[:,1]

AttributeError: 'RidgeClassifierCV' object has no attribute 'predict_proba'

# I found the Ridge Classifier, but I don't see an equivalent for Lasso.  Can I do Lasso for a binary classification problem?

### Logistic Regression

In [ ]:
from sklearn import linear_model
logreg = LogisticRegressionCV()
logreg.fit(X_train, y_train)

### K Nearest Neighbors Classifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(iris['data'], iris['target'])
#see the example about param grid - it is using grid search to determine the best number of neighbors

In [ ]:
#param_grid = {'n_neighbors': np.arange(1, 50)}
#knn = KNeighborsClassifier()
#knn_cv = GridSearchCV(knn, param_grid, cv=5)
#knn_cv.fit(X, y)
#knn_cv.best_params_
#knn_cv.best_score_

In [ ]:
In [7]: from sklearn.model_selection import cross_val_score
In [8]: cv_scores = cross_val_score(logreg, X, y, cv=5,
...: scoring='roc_auc')
In [9]: print(cv_scores)
[ 0.99673203 0.99183007 0.99583796 1. 0.96140652]

### Support Vector Classifier

In [ ]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

### Random Forest

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

### Catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

train_data = Pool(data=[[1, 4, 5, 6],
                        [4, 5, 6, 7],
                        [30, 40, 50, 60]],
                  label=[1, 1, -1],
                  weight=[0.1, 0.2, 0.3])

model = CatBoostClassifier(iterations=10)

model.fit(train_data)
preds_class = model.predict(train_data)

# Roc Auc Score

In [ ]:
logreg = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
logreg.fit(X_train, y_train)
y_pred_prob = logreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

In [ ]:
pca = PCA()
y_pca = pca.fit_transform(df_y_with_dummies)

In [ ]:
inc_pca = IncrementalPCA()
#inc_pca = IncrementalPCA(n_components=100,copy=False,batch_size=10)
# y_pca = inc_pca.fit_transform(df_y_with_dummies)
inc_pca.fit(df_y_with_dummies)
y_pca = inc_pca.transform(df_y_with_dummies)

In [ ]:
tsvd = TruncatedSVD()
y_tsvd = tsvd.fit_transform(df_y_with_dummies)